In [ ]:
import awswrangler  as wr
import pandas as pd
import pymysql
import datetime
import json
import requests

def lambda_handler(event, context):
    
    sql_query = """
    with max_date_df as (
       select max(date_ymd) as max_date FROM "moyo_mart"."plan_ranking_score"
    ),
    
    rankings as (SELECT * 
    FROM "moyo_mart"."plan_ranking_score"
    where date_ymd = (select max_date from max_date_df)),
    
    delayed_mvno as (SELECT * 
    FROM "moyo_mart"."delayed_mvno"
    where date_ymd = (select max_date from max_date_df)),
    
    mpcm as (select distinct mvno
    from moyo_data.mobile_plan_contract_mvno)
    
    select distinct *, 
          round(percent_rank() over (order by popularity_score_with_mvno_score) * 100,2)  as popularity_score_with_mvno_score_pct,
          round(percent_rank() over (order by popularity_score_with_mvno_score_v2) * 100,2)  as popularity_score_with_mvno_score_pct_v2
    from
    (select plan_id,
           is_ban as is_delayed,
           total_score as popularity_score,
           total_score_v2 as popularity_score_v2,
           round(percent_rank() over (order by total_score) * 100,2) as popularity_score_pct,
           round(percent_rank() over (order by total_score_v2) * 100,2) as popularity_score_pct_v2,
           round(percent_rank() over (order by delayed_order_ratio desc) * 100,2) as mvno_score,
           total_score_v2 + 0.01 * percent_rank() over (order by delayed_order_ratio desc) as popularity_score_with_mvno_score,
           total_score_v2 + 0.01 * percent_rank() over (order by delayed_order_ratio desc) as popularity_score_with_mvno_score_v2,
           fee_score,
           spec_score,
           fee_after_discount_score,
           discount_period_score,
           case when applied_completed_date <= 3 then 3
           when applied_completed_date >= 15 then 15
           else applied_completed_date end as completed_date
    from
    (select a.id as plan_id,
           a.name as plan_name,
           a.mvno,
           a.mno,
           case when c.mvno is not null then '제휴사' else '비제휴사' end as is_contract,
           mobile_data,
           mobile_data2,
           speed_when_exhausted,
           voice,
           message,
           fee,
           fee_after_discount,
           discount_period,
           total_score,
           total_score_v2,
           round(pct_fee * 100) as fee_score,
           round(pct_order_count * 100) as spec_score,
           round(pct_fee_after_discount * 100) as fee_after_discount_score,
           round(pct_discount_period * 100) as discount_period_score,
           case when delayed_order_ratio is null and c.mvno is not null then (select approx_percentile(delayed_order_ratio,0.75) from delayed_mvno)
           when delayed_order_ratio is null and c.mvno is null then (select approx_percentile(delayed_order_ratio,1) from delayed_mvno)
           else delayed_order_ratio end as delayed_order_ratio,
           case when delayed_order_ratio >= 70 then 1 else 0 end as is_ban,
           case when applied_completed_date is null and c.mvno is not null then (select approx_percentile(applied_completed_date,0.5) from delayed_mvno)
           when applied_completed_date is null and c.mvno is null then (select approx_percentile(applied_completed_date,0.8) from delayed_mvno)
           else applied_completed_date end as applied_completed_date
    from rankings a
    left outer join delayed_mvno b
    on a.mvno = b.mvno and a.mno = b.mno
    left outer join mpcm c
    on a.mvno = c.mvno))
    order by popularity_score_with_mvno_score_v2 desc
    """

    df = wr.athena.read_sql_query(sql=sql_query, database="moyo_logs")


    result = json.loads(df.to_json(orient = 'records'))
    result = json.dumps(result, ensure_ascii = False)
    result = str(result).replace('/','')
    result = json.loads(result)
    return result

In [6]:
import awswrangler  as wr
import pandas as pd
import pymysql
import datetime
import json
import requests

def lambda_handler(event, context):
        
    sql_query = """
    select 
    object_id 
    , navigation 
    , object_section 
    from moyo_logs.user_behavior_logs
    where 1=1   
        and date_ymd = '2023-05-01'
    limit 10 
    """

    df = wr.athena.read_sql_query(sql=sql_query, database="moyo_logs")


    result = json.loads(df.to_json(orient = 'records'))
    result = json.dumps(result, ensure_ascii = False)
    result = str(result).replace('/','')
    result = json.loads(result)

    return  result

In [4]:
print( result )

[{'object_id': '14488', 'navigation': 'search_plans', 'object_section': 'plan_list'}, {'object_id': '13903', 'navigation': 'search_plans', 'object_section': 'recently_plans'}, {'object_id': '10967', 'navigation': 'search_plans', 'object_section': 'plan_list'}, {'object_id': '8089', 'navigation': 'search_plans', 'object_section': 'plan_list'}, {'object_id': '3', 'navigation': 'search_plans', 'object_section': 'plan_list'}, {'object_id': '10326', 'navigation': 'search_plans', 'object_section': 'recently_plans'}, {'object_id': '10326', 'navigation': 'search_plans', 'object_section': 'recently_plans'}, {'object_id': '13796', 'navigation': 'search_plans', 'object_section': 'plan_list'}, {'object_id': '8200', 'navigation': 'search_plans', 'object_section': 'plan_list'}, {'object_id': '14492', 'navigation': 'search_plans', 'object_section': 'plan_list'}]


In [13]:
import pandas as pd

sheet_id = '1yiBeHSzHF_aCFJr28XHjpNKvekPOsfUvER15Jm4Ssn4'
sheet_id2 = '1IcZ_tQ67i0C9-Je4jFyMmbCqQzFgyQiojYos0kNDHzA'

df = pd.read_csv(f'https://docs.google.com/spreadsheets/d/{sheet_id2}/export?format=csv')

In [14]:
# https://docs.google.com/spreadsheets/d/1IcZ_tQ67i0C9-Je4jFyMmbCqQzFgyQiojYos0kNDHzA/edit#gid=1686811376
print(df)

    <!doctype html><html lang="en-US" dir="ltr"><head><base href="https://accounts.google.com/v3/signin/"><meta name="referrer" content="origin"><meta name="viewport" content="width=device-width  \
0    /*# sourceURL=/_/mss/boq-identity/_/ss/k=boq-i...                                                                                                                                               
1                         (function(){var p=function(L                                                                                                                                               
2                        '(function(){var w=function(X                                                                                                                                               
3               Copyright The Closure Library Authors.                                                                                                                                               
4         

https://docs.google.com/spreadsheets/d/1yiBeHSzHF_aCFJr28XHjpNKvekPOsfUvER15Jm4Ssn4/edit#gid=0